In [15]:
from bs4 import BeautifulSoup
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
os.environ["LANG"] = "en_US.UTF-8"

In [16]:
#read in the links master list
import csv

master_list = []

file = open("./Downloads/urls.csv", "r")
reader = csv.reader(file)
for row in reader:
    master_list.append(row[-1])
file.close()

# remove the column header
master_list.pop(0)

'URL'

In [17]:

ff_options = Options()
ff_options.add_argument("--headless")

driver = webdriver.Firefox(options=ff_options)

def collect_lesson_links(base_url):
    links = []
    try:
        driver.get(base_url)
        # Wait for dynamic content to load    
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="/lesson/"]'))
        )
        # Collect all links that begin with "/lesson/"
        links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[href^="/lesson/"]')]
        # Make links absolute
        links = [f"https://llis.nasa.gov{link}" if link.startswith('/lesson/') else link for link in links]
    except:
        print("Error getting links from: " + base_url)
    return links

def scrape_lesson_content(links):
    data = []
    for link in links:
        print("Reading: " + link)
        try:
            driver.get(link)
            # Allow some time for the page to load and JavaScript to render content
            time.sleep(5)
            # Use BeautifulSoup to parse page source and extract <p> text
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            paragraphs = soup.find_all('p')
            for p in paragraphs:
                # Ensure that paragraph text is not empty or repetitive
                paragraph_text = p.text.strip()
                if paragraph_text and paragraph_text not in [row[1] for row in data]:
                    data.append([link, paragraph_text])
        except:
            print("Error reading from: " + link)
    return data

In [18]:
# Scrape content from each link
content = scrape_lesson_content(master_list)

Reading: https://llis.nasa.gov/lesson/0740.html
Reading: https://llis.nasa.gov/lesson/1
Reading: https://llis.nasa.gov/lesson/10
Reading: https://llis.nasa.gov/lesson/100
Reading: https://llis.nasa.gov/lesson/1000
Reading: https://llis.nasa.gov/lesson/10001
Reading: https://llis.nasa.gov/lesson/1001
Reading: https://llis.nasa.gov/lesson/1002
Reading: https://llis.nasa.gov/lesson/1003
Reading: https://llis.nasa.gov/lesson/1004
Reading: https://llis.nasa.gov/lesson/1005
Reading: https://llis.nasa.gov/lesson/1006
Reading: https://llis.nasa.gov/lesson/1007
Reading: https://llis.nasa.gov/lesson/1008
Reading: https://llis.nasa.gov/lesson/1009
Reading: https://llis.nasa.gov/lesson/101
Reading: https://llis.nasa.gov/lesson/1010
Reading: https://llis.nasa.gov/lesson/1011
Reading: https://llis.nasa.gov/lesson/1012
Reading: https://llis.nasa.gov/lesson/1013
Reading: https://llis.nasa.gov/lesson/1014
Reading: https://llis.nasa.gov/lesson/1015
Reading: https://llis.nasa.gov/lesson/1016
Reading: htt

TimeoutException: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:740:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19


In [ ]:
# Convert the list to a DataFrame
df = pd.DataFrame(content, columns=['URL', 'Paragraph'])

# Save the DataFrame to a CSV file
df.to_csv('./Downloads/lesson_content.csv', index=False)

driver.quit()